### Setup for easy experimenation

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
os.getcwd()

'/Users/pradip.gupta/personal-projects/grab/notebooks'

In [3]:
os.chdir(os.path.dirname(os.getcwd()))
os.getcwd()

'/Users/pradip.gupta/personal-projects/grab'

## Problem Statement for the Competition:

As presented here: https://www.aiforsea.com/computer-vision

### Given a dataset of distinct car images, can you automatically recognize the a) `car model` and b) `car make`?

The problem statement talks about identifying car model and car make. So, there are 2 taget variables here. In the video presented [here](https://www.youtube.com/watch?v=7BL8EeAkNDw&feature=youtu.be&t=92) even `c) dominate color prediction` was posed as part of this problem, however, as that is not a part of the problem statement, I left that third component out.

## What is Car Make and Car Model?

Lets understand what does these two terms mean: <br>

A car's make is the brand of the vehicle, while the model refers to the name of a car product and sometimes a range of products. For example, Toyota is a car make and Camry is a car model. 

## About the data:

Source: https://ai.stanford.edu/~jkrause/cars/car_dataset.html

- The Cars dataset contains **16,185** images of **196 classes** of cars. 

- The data is split into **8,144 training** images and **8,041 testing** images, where each class has been split roughly in a 50-50 split. 

- Classes are typically at the level of Make, Model, Year, e.g. 2012 Tesla Model S or 2012 BMW M3 coupe.

More about the dataset @: https://ai.stanford.edu/~jkrause/papers/3drr13.pdf

## 1. Class Seperation:

The intial thought was to treat the 2 target variables individually, and make a multi-task learning model, that can produce outputs for the 2 classes individually. However, as the hold-out data is not availabel, it was difficult to visualise what label structure the hold out data is following, so the idea of considering the output as two independent labels was **dropped**.

However, I have presented the work here. Please note this seperation has **not** been used in the actually modelling. 

## A brief study on different car models that exists:

1. Hatchback, 
2. Sedan, 
3. MPV, 
4. SUV, 
5. Crossover, 
6. Coupe, 
7. Convertible, 
8. Truck, 
9. Van, 
10. Wagon,
12. Sports car
13. Diesel, 
14. Luxury car
15. Electric

others....

In [5]:
import pandas as pd

`cars_meta.mat`: Contains a cell array of class names, one for each class.

In [ ]:
labels = mat_io.loadmat(preprocessing["labels"])

In [ ]:
data = []

for i, label in enumerate(labels["class_names"][0]):
    
    label = label[0]
    make = label.split()[0]
    descr = ' '.join(label.split()[1:-1])
    model = label.split()[-2]
    
    if "Martin" in model:
        make = "Aston Martin"
        descr = descr.replace("Martin","")
    
    if make == "AM" and "General" in descr:
        make = "AM General"
        descr = descr.replace("General", "")
        
    if descr == 'Integra Type R':
         model = 'Type-R'

    if model == 'Z06' or model == 'ZR1':
        model = 'Convertible'

    if 'SRT' in model:
        model = 'SRT'

    if model == 'IPL':
        model = 'Coupe'

    year = label.split()[-1]
    data.append((i, label, make, descr, model, year))

df = pd.DataFrame(data, columns=['target', 'label', 'make', 'description', 'model', 'year'])

In [ ]:
df.head()

In [ ]:
df.make.unique(), df.make.nunique()

In [ ]:
df.model.unique(), df.model.nunique()

In [ ]:
condition = df["model"]=="SS"
desired_col = ["label"]
df.loc[condition,desired_col]

### With this exercise I could have defined 2 variables as make and model with 49 and 18 classes respectively. However, as the nature of Hold-out data was not known I skipped this step. 

## 2. Defining ROI for Input images:

To improve the classification accuracy, it is desired that we restric our AOI to the car region only and remove other background in the image which may act as noise and prevent the model from learning the right features.

In [6]:
#Standard imports
from scipy import io as mat_io
import numpy as np

In [8]:
#Custom imports
from src2.config import preprocessing
preprocessing

{'labels': 'data/devkit/cars_meta.mat',
 'train_annotations': 'data/devkit/cars_train_annos.mat',
 'test_annotations': 'data/devkit/cars_test_annos.mat',
 'raw_train_images': 'data/raw/cars_train',
 'raw_test_images': 'data/raw/cars_train',
 'extracted_train_images': 'data/processed/cars_train'}

In [9]:
train_annotations = preprocessing["train_annotations"]
raw_train_images = preprocessing["raw_train_images"]
processed_train_images = preprocessing["extracted_train_images"]

In [10]:
labels = mat_io.loadmat(preprocessing["labels"])

In [18]:
class_dict = {}
class_names = []
class_ids = []

for i, label in enumerate(labels["class_names"][0]):
    class_dict[i] = label[0]
    class_names.append(label[0])
    class_ids.append(i)

In [13]:
annoations_dict = mat_io.loadmat(train_annotations)

As given at the datasource:

`cars_train_annos.mat`: <br>

Contains the variable 'annotations', which is a struct array of length num_images and where each element has the fields:

- `bbox_x1:` Min x-value of the bounding box, in pixels
- `bbox_x2:` Max x-value of the bounding box, in pixels
- `bbox_y1:` Min y-value of the bounding box, in pixels
- `bbox_y2:` Max y-value of the bounding box, in pixels
- `class:` Integral id of the class the image belongs to.
- `fname:` Filename of the image within the folder of images.

In [16]:
files_list = []
for annotation in annoations_dict['annotations'][0]:
    
    #path/to/image.jpg,x1,y1,x2,y2,class_name
    
    class_id = annotation["class"][0][0] - 1
    
    value = (annotation["fname"][0], annotation["bbox_x1"][0][0], annotation["bbox_y1"][0][0],
            annotation["bbox_x2"][0][0], annotation["bbox_y2"][0][0], class_dict[class_id])
    files_list.append(value)

In [24]:
def class_map_csv(class_names, class_ids):    
    class_mapping = pd.DataFrame({"class_names":class_names,
                     "class_ids":class_ids})
    
    return class_mapping

In [25]:
column_name = ['filename', 'xmin', 'ymin', 'xmax', 'ymax', 'class']
df = pd.DataFrame(files_list, columns=column_name) 
    
msk = np.random.rand(len(df)) < 0.8
train_df = df[msk]
val_df = df[~msk]
    
train_df.to_csv(raw_train_images+'/train_annotations.csv', header=False, index=None)
val_df.to_csv(raw_train_images+'/val_annotations.csv', header=False, index=None)
    
class_mapping = class_map_csv(class_names, class_ids)
class_mapping.to_csv(raw_train_images+"/class_mapping.csv",header=False, index=False)

In [28]:
#check if all images were loaded properly
assert df.shape[0] == 8144

8144